In [87]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchinfo import summary

- We adopt batch normalization (BN) right after each convolution and before activation. <br></br>
- We initialize the weights as in Kaming He initialization and train all plain/residual nets from scratch.

- Option B is used for shortcut connection

<img src="./table3.png" width="450" height="450"/>

<img src="./identity_mapping.png" width="450" height="450"/>

### Block implementation

<img src='./Blocks.png' width='550' height='350'/>

In [144]:
class BasicBlock(nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel, kernel_size=3, stride=1, padding=1, bias=False)

        self.bn1 = nn.BatchNorm2d(out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)

        self.residual_function = nn.Sequential(self.conv1, self.bn1, nn.ReLU(inplace=True), self.conv2, self.bn2)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channel != out_channel:
            self.shortcut = nn.Sequential(nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=1, stride=stride, padding=0, bias=False), nn.BatchNorm2d(out_channel))
        
    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = F.relu(x)

        return x

In [145]:
class BottleNeck(nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(BottleNeck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=1, stride=1, padding=0, bias=False)
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv3 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel * 4, kernel_size=1, stride=1, padding=0, bias=False)

        self.bn1 = nn.BatchNorm2d(out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.bn3 = nn.BatchNorm2d(out_channel * 4)

        self.residual_function = nn.Sequential(self.conv1, self.bn1, nn.ReLU(inplace=True), self.conv2, self.bn2, nn.ReLU(inplace=True), self.conv3, self.bn3)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channel != out_channel * 4:
            self.shortcut = nn.Sequential(nn.Conv2d(in_channels=in_channel, out_channels=out_channel * 4, kernel_size=1, stride=stride, padding=0, bias=False), nn.BatchNorm2d(out_channel * 4))
        
    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = F.relu(x)

        return x

<img src='./architecture.png' height='350' width='650'>

Parameters : num_blocks, Blocks ( Basic Block (18, 34) // BottleNeck (50, 101, 152))

In [146]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks):
        super(ResNet, self).__init__()

        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.conv2_x = self._make_layer(block, out_channels=64, num_block=num_blocks[0], stride=1)
        self.conv3_x = self._make_layer(block, out_channels=128, num_block=num_blocks[1], stride=2)
        self.conv4_x = self._make_layer(block, out_channels=256, num_block=num_blocks[2], stride=2)
        self.conv5_x = self._make_layer(block, out_channels=512, num_block=num_blocks[3], stride=2)

        self.avg_pool = nn.AvgPool2d(kernel_size=(7, 7))

        if block.__name__ == 'BasicBlock':
            self.fc1 = nn.Linear(in_features=512, out_features=1000)
        else:
            self.fc1 = nn.Linear(in_features=2048, out_features=1000)

    def _make_layer(self, block, out_channels, num_block, stride):
        strides = [stride] + [1] * (num_block - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            if block.__name__ == 'BasicBlock':
                self.in_channels = out_channels
            else:
                self.in_channels = out_channels * 4
        print(layers)
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        print(self.conv2_x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)

        x = self.avg_pool(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)

        return x

In [149]:
summary(ResNet(BasicBlock, [3,4,6,3]), (32, 3, 224, 224))

[BasicBlock(
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (residual_function): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (shortcut): Sequential()
), BasicBlock(
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [32, 1000]                --
├─Sequential: 1-1                        [32, 64, 56, 56]          --
│    └─Conv2d: 2-1                       [32, 64, 112, 112]        9,408
│    └─BatchNorm2d: 2-2                  [32, 64, 112, 112]        128
│    └─ReLU: 2-3                         [32, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                    [32, 64, 56, 56]          --
├─Sequential: 1-2                        [32, 64, 56, 56]          --
│    └─BasicBlock: 2-5                   [32, 64, 56, 56]          --
│    │    └─Sequential: 3-1              [32, 64, 56, 56]          73,984
│    │    └─Sequential: 3-2              [32, 64, 56, 56]          --
│    └─BasicBlock: 2-6                   [32, 64, 56, 56]          --
│    │    └─Sequential: 3-3              [32, 64, 56, 56]          73,984
│    │    └─Sequential: 3-4              [32, 64, 56, 56]          --
│  

In [150]:
from torchvision.models import resnet34

In [151]:
model = resnet34()

In [152]:
summary(model, (32, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [32, 1000]                --
├─Conv2d: 1-1                            [32, 64, 112, 112]        9,408
├─BatchNorm2d: 1-2                       [32, 64, 112, 112]        128
├─ReLU: 1-3                              [32, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [32, 64, 56, 56]          --
├─Sequential: 1-5                        [32, 64, 56, 56]          --
│    └─BasicBlock: 2-1                   [32, 64, 56, 56]          --
│    │    └─Conv2d: 3-1                  [32, 64, 56, 56]          36,864
│    │    └─BatchNorm2d: 3-2             [32, 64, 56, 56]          128
│    │    └─ReLU: 3-3                    [32, 64, 56, 56]          --
│    │    └─Conv2d: 3-4                  [32, 64, 56, 56]          36,864
│    │    └─BatchNorm2d: 3-5             [32, 64, 56, 56]          128
│    │    └─ReLU: 3-6                    [32, 64, 56, 56]          --
│